In [29]:
import os
from core.utils import create_k_fold_splits
sep = os.sep
import numpy as np
from torch.utils.data import Dataset, DataLoader



In [22]:
sites = [0, 1 ,2, 3]

In [23]:
# for site in sites:
#     pth = f'test/input/local{site}/simulatorRun'
#     create_k_fold_splits(os.listdir(pth+sep+f'nifti_file_S0{site}'), k=10,
#                          save_to_dir=pth+sep+f'split_S0{site}')

In [24]:
import nibabel as ni
import torchvision.transforms as tmf

In [25]:
nf = ni.load('test/input/local0/simulatorRun/nifti_file_S00/M02101222_swc1t1avg.nii').dataobj

In [43]:
nf = np.array(nf)

In [45]:
nf.shape

(121, 145, 121)

In [47]:
nf[None, :].shape

(1, 121, 145, 121)

In [28]:
class NiftiDataset(Dataset):
    def __init__(self, **kw):
        self.files_dir = kw['files_dir']
        self.labels_dir = kw['labels_file']
        self.mode = kw['mode']
        self.indices = []

    def load_indices(self, files, **kw):
        labels_file = os.listdir(self.labels_dir)[0]
        labels = pd.read_csv(self.labels_dir + os.sep + labels_file).set_index('niftifile')
        for file in files:
            y = labels.loc[file]['label']
            """
            int64 could not be json serializable.
            """
            self.indices.append([file, int(y)])

    def __getitem__(self, ix):
        file, y = self.indices[ix]
        nif = np.array(ni.load(self.files_dir + sep + file).dataobj)
        return {'inputs': torch.tensor(nif), 'labels': torch.tensor(y)}

    def __len__(self):
        return len(self.indices)
    
    def get_transformations(self):
        return tmf.Compose([tmf.RandomHorizontalFlip, tmf.RandomVerticalFlip])
        

    def get_loader(self, shuffle=False, batch_size=None, num_workers=0, pin_memory=True, **kw):
        return NNDataLoader.get_loader(dataset=self, shuffle=shuffle, batch_size=batch_size,
                                       num_workers=num_workers, pin_memory=pin_memory, **kw)
    
    


In [42]:
import torch.nn as nn


class MSANNet(nn.Module):
    def __init__(self, in_size, hidden_sizes, out_size, dropout_in=[]):
        super(MSANNet, self).__init__()
        self.in_size = in_size
        self.out_size = out_size
        self.hidden_sizes = hidden_sizes
        self.layers = nn.ModuleList()

        current_dim = self.in_size
        for i, h_dim in enumerate(hidden_sizes):
            if i in dropout_in:
                layers = [nn.Linear(current_dim, h_dim, bias=False), nn.BatchNorm1d(h_dim), nn.ReLU(),
                          nn.Dropout(p=0.5)]
            else:
                layers = [nn.Linear(current_dim, h_dim, bias=False), nn.BatchNorm1d(h_dim), nn.ReLU()]

            self.layers.append(nn.Sequential(*layers))
            current_dim = h_dim

        self.fc_out = nn.Linear(current_dim, out_size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return self.fc_out(x)
